In [1]:
%matplotlib inline
## imports

# site
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import affine
import flopy

# std
from pathlib import Path

In [2]:
def read_array(rasterfile, masked=True,  band=1):
    with rasterio.open(rasterfile) as src:
        return src.read(band, masked=masked)

In [3]:
## input

# name
name = 'test_wel_brabant'

# workspace
workspace = Path(r'.\output\wel')
                 
# data file
sqfile = Path(r'..\mf6\data\wells\sq_list.csv')

In [4]:
ncols=       601
nrows=       450
xllcorner=   60000.
yllcorner=   322500.
cellsize=    250.

# Affine transformation is a lineair transformation that preserves parallel lines
# To do: incorporate corner coordinates in DIS
geotransform = (xllcorner, cellsize, 0.0, yllcorner, 0.0, cellsize)

# to row,col from x,y
fwd = affine.Affine.from_gdal(*geotransform);

# to x,y from row,col
rev = ~fwd

In [10]:
# read sources data from csv file
sqs = pd.read_csv(sqfile)[['id','xcoordinate','ycoordinate', 'ilay', 'q_assigned','SRCID']]

# layer numbering from Q3D to 3D
sqs['ilay'] = sqs['ilay'].apply(lambda x: x*2-1)

# transform xy to row,col
sqs['xcoordinate'],sqs['ycoordinate'] = rasterio.transform.rowcol(fwd, sqs['xcoordinate'], sqs['ycoordinate'])
sqs['spdata'] = sqs[['ilay','xcoordinate','ycoordinate']].values.tolist()

In [6]:
# create workspace directory
workspace.mkdir(exist_ok=True)

In [ ]:
wel = flopy.mf6.modflow.mfgwfwel.ModflowGwfwel(gwf,
    pname='wel', 
    stress_period_data=(sqs['spdata'],sqs['q_assigned'],sqs['SRCID']))